<a href="https://colab.research.google.com/github/BanSangSu/Huggingface-AI-Agents-course/blob/main/unit2/langgraph/agent_with_multiple_tools_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import & API keys

In [ ]:
%pip install -U rank_bm25 datasets langchain langchain_community langchain_openai langgraph langchain_core duckduckgo-search pyowm

In [ ]:
# huggingface api
from huggingface_hub import login
login()

In [ ]:
import os
from getpass import getpass

# weather api
os.environ["OPENWEATHERMAP_API_KEY"] = getpass()

# llm api
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")
os.environ["OPENAI_API_BASE"] = getpass("OpenAI OPENAI BASE URL:")

## Tools
(You can use these tools independently by creating standalone Python (.py) modules)

### Retriever Tool

In [ ]:
import datasets
from langchain_core.documents import Document

guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]

from langchain_community.retrievers import BM25Retriever
from langchain.tools import Tool

bm25_retriever = BM25Retriever.from_documents(docs)

def extract_text(query: str) -> str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.invoke(query)
    if results:
        return "\n\n".join([doc.page_content for doc in results[:3]])
    else:
        return "No matching guest information found."

guest_info_tool = Tool(
    name="guest_info_retriever",
    description="Retrieves detailed information about gala guests based on their name or relation.",
    func=extract_text
)

### Web search, Weather information & Hub Stats tools

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [ ]:
from langchain.tools import Tool
from langchain_community.utilities import OpenWeatherMapAPIWrapper

def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location.
    You have to type city,Country Code . ex) Seoul,KR """
    weather = OpenWeatherMapAPIWrapper()

    weather_data = weather.run(location)

    return weather_data


weather_info_tool = Tool(
    name="get_weather_info",
    func=get_weather_info,
    description="Fetches dummy weather information for a given location. You have to type city,Country Code . ex) Seoul,KR "
)

In [ ]:
from langchain.tools import Tool
from huggingface_hub import list_models

def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the Hugging Face Hub."""
    try:
        models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

        if models:
            model = models[0]
            return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
        else:
            return f"No models found for author {author}."
    except Exception as e:
        return f"Error fetching models for {author}: {str(e)}"


hub_stats_tool = Tool(
    name="get_hub_stats",
    func=get_hub_stats,
    description="Fetches the most downloaded model from a specific author on the Hugging Face Hub."
)

## The Complete Agent

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition

from langchain_openai import ChatOpenAI

## When you import tools from standalone python files.
# from tools import DuckDuckGoSearchRun, weather_info_tool, hub_stats_tool
# from retriever import guest_info_tool

### Set the LLM model

In [ ]:
model = getpass("Model:")

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=model,
    temperature=0
)

tools = [guest_info_tool, search_tool, weather_info_tool, hub_stats_tool]
llm_with_tools = llm.bind_tools(tools)

### Build LangGraph

In [ ]:
from typing import Dict, List, Any

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState) -> Dict[str, List[Any]]:
    return {
        "messages": [llm_with_tools.invoke(state["messages"])],
    }

builder = StateGraph(AgentState)

builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

## Using tool examples

In [ ]:
response = alfred.invoke({"messages": "Tell me about 'Lady Ada Lovelace'"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

In [ ]:
response = alfred.invoke({"messages": "What's the weather like in Paris tonight? Will it be suitable for our fireworks display?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

In [ ]:
response = alfred.invoke({"messages": "One of our guests is from Qwen. What can you tell me about their most popular model?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

In [ ]:
response = alfred.invoke({"messages":"I need to speak with 'Dr. Nikola Tesla' about recent advancements in wireless energy. Can you help me prepare for this conversation?"})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

### Conversation Memory

In [ ]:
response = alfred.invoke({"messages": [HumanMessage(content="Tell me about 'Lady Ada Lovelace'. What's her background and how is she related to me?")]})


print("🎩 Alfred's Response:")
print(response['messages'][-1].content)
print()

response = alfred.invoke({"messages": response["messages"] + [HumanMessage(content="What projects is she currently working on?")]})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)